# Homography

In [2]:
%matplotlib notebook

import matplotlib
import ipywidgets as wdg

from machinevisiontoolbox import Image, CentralCamera
import matplotlib.pyplot as plt
import numpy as np
from spatialmath import SE3, SO3
from spatialmath import base

from unknown_camera import CreateSimulatedCamera

np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

In this notebook we are going to investigate the homography between the image plane and a ground plane, for the case of a camera attached to a mobile robot.

## Simulate a Camera

The relevant coordinate frames are as shown here:


![frames](images/coordinate-frames.png)

{R} is the robot frame, sitting on the ground with its z-axis upwards, and the camera is above the ground with its z-axis pointing forward.

The orientation of the camera is given by a roll angle of 3° and a pitch angle of -92° to simulate non-perfect attachment of the camera to the robot, ie. the camera is pitched slightly toward the floor and rotated about it's optical axis.

In [7]:
R = SO3.Rx(-92, 'deg') * SO3.Rz(2, 'deg')
T = SE3([0, 0, 0.02]) * SE3.SO3(R)
camera = CentralCamera(imagesize=(1280,1024), f=0.015, pose=T)

principal point not specified,                    setting it to centre of image plane


## Mapping Ground Plane Coordinates to the Image Plane

Consider a point on the ground in front of the robot

In [10]:
P1 = [0, 0.5, 0]

It would be projected to the image plane at

In [11]:
camera.project(P1)

array([[   640.3],
       [   519.6]])

Choose another point, further from the robot

In [13]:
P2 = [0, 0.6, 0]
camera.project(P2)

array([[   639.9],
       [   509.6]])

and we see it is projected ~10 pixels higher in the image.

We note that the u-coordinate of both points is close to the u-coordinate of the principal point (640), the centre of the image, which indicates that the world point is close to the optical axis (as the x-coordinate of 0 would indicate).  Remember that the camera is slightly skewed with respect to the robot frame.

The camera matrix is

In [14]:
camera.C

array([[    1499,    637.8,   -74.65,    1.493],
       [  -52.35,    459.4,    -1516,    30.32],
       [       0,   0.9994,  -0.0349, 0.000698]])

## Mapping image plane coordinates to the ground plane

In the previous section we intuitively worked through how we can map ground points into the image plane. Now we will see how can we do the opposite. 

Points on the ground plane have a z-coordinate of zero.  The camera projection equation, in matrix form, is
$$
\tilde{p} = \begin{pmatrix} c_1 & c_2 & c_3  & c_4 \end{pmatrix} \begin{pmatrix} X \\ Y \\ Z \\ 1 \end{pmatrix}
$$
where $c_i$ is the $i^{th}$ column of the camera matrix.  Since $Z=0$ we can rewrite as 

\begin{align}
\tilde{p} & = \begin{pmatrix} c_1 & c_2 & c_4 \end{pmatrix} \begin{pmatrix} X \\ Y \\ 1 \end{pmatrix} \\
          & = \mathbf{H} \begin{pmatrix} X \\ Y \\ 1 \end{pmatrix}
\end{align}
where $(X, Y)$ is a point on the ground plane with respect to the camera frame which has the Y-axis forward and the X-axis to the right.

In this case $\mathbf{H}$ is

In [15]:
H = camera.C[:,[0,1,3]]
H

array([[    1499,    637.8,    1.493],
       [  -52.35,    459.4,    30.32],
       [       0,   0.9994, 0.000698]])

which is both square and non-singular

In [16]:
np.linalg.det(H)

-44999.999999999985

This means we have an invertible mapping between 2D points in the image plane and 2D points on the ground plane, both expressed in homogenous form.  Such a transformation is called an *homography*.

**Note that in general we cannot map a 2D point to a 3D point, since a 2D image plane coordinate corresponds to a ray in space -- an infinite number of points.  However the constraint that the point lies on the ground plane makes a unique solution possible, it is the point where that ray intersects the ground plane.**

Consider a point in the image plane at $(300, 600)$

In [22]:
p = (300, 600)  # make homogeneous
Pg = base.h2e(np.linalg.inv(H) @ base.e2h(p))
print(Pg)

[[-0.05341]
 [  0.2331]]


and this is corresponding point coordinate on the ground plane. x=-0.027 indicates it is to the left of the robot, and y=0.117 indicates it is in-front of the robot.

We can cross check this by reprojecting that world point back to the image plane using our camera model

In [23]:
P = np.vstack((Pg, 0))  # point is on the ground, add Z=0
print(P)
camera.project(P)

[[-0.05341]
 [  0.2331]
 [       0]]


array([[     300],
       [     600]])

and we obtain, as we should do, the image plane coordinate that we started with.